In [2]:
import numpy
import pandas
#import keras

In [25]:
#Wenjie's 
#ds_test = pandas.read_csv("we_data/test.csv")
#ds_train = pandas.read_csv("we_data/train.csv")
#ds_val = pandas.read_csv("we_data/validation.csv")

In [35]:
#Aisha's 
#ds_test = pandas.read_csv("/Users/aisha/desktop/AI/we_data/test.csv\")
#ds_train = pandas.read_csv("/Users/aisha/desktop/AI/we_data/train.csv\")
#ds_val = pandas.read_csv("/Users/aisha/desktop/AI/we_data/validation.csv\")

In [4]:
# Hend's
ds_test = pandas.read_csv("/Users/HendAlmalik/Documents/Term2/AI/CW/we_data/test.csv")
ds_train = pandas.read_csv("/Users/HendAlmalik/Documents/Term2/AI/CW/we_data/train.csv")
ds_val = pandas.read_csv("/Users/HendAlmalik/Documents/Term2/AI/CW/we_data/validation.csv")

# Dataset Info

## Training Data

In [5]:
df = ds_train

print('headers:', list(df))

print('size:', len(df))
df.head()

headers: ['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag']
size: 2430981


,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,...,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,NaN
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,...,250,FourthView,Na,0,10722,294,23,NaN,2821,NaN
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,...,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110"
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111"
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,...,90,OtherView,Na,133,7330,277,133,NaN,2259,NaN


In [6]:
# Stats on impressions with clicks
max = df[df['click'] != 0]['payprice'].max()
min = df[df['click'] != 0]['payprice'].min()
x = df[df['click'] != 0]['payprice'].mean()
y = df[df['click'] != 0]['payprice'].std()
print(f"Mean {x}, std {y}, mean+std {x+y}, max {max}, min {min}")

Mean 105.46402677077523, std 66.39225768437046, mean+std 171.8562844551457, max 300, min 4


In [8]:
# Stats on bidprice and payprice for all impressions
for x in ['bidprice', 'payprice']:
    print('Average {0}: {1}'.format(x, df[x].mean()))
    print('Lowest {0}: {1}'.format(x, df[x].min()))
    print('Highest {0}: {1}'.format(x, df[x].max()))

Average bidprice: 272.9620461862927
Lowest bidprice: 227
Highest bidprice: 300
Average payprice: 78.15141623895867
Lowest payprice: 0
Highest payprice: 300


In [7]:
###### Range of the Random Strategy ######
for x in ['slotprice']:
    print('Average {0}: {1}'.format(x, df[x].mean()))
    print('Lowest {0}: {1}'.format(x, df[x].min()))
    print('Highest {0}: {1}'.format(x, df[x].max()))

Average slotprice: 26.71782502619313
Lowest slotprice: 0
Highest slotprice: 300


In [8]:
# Selection of useful fields per advertise
tmp1 = df[['click', 'bidprice', 'payprice', 'advertiser']].groupby('advertiser').sum()
# Calcuate impressions per advertiser
tmp2 = df[['click', 'bidprice', 'payprice', 'advertiser']].groupby('advertiser').size().reset_index(name='impressions')

# Joining the 2 dataframes
stats = tmp1.join(tmp2.set_index('advertiser'), on='advertiser')

# Calculate Click-Through Rate (CTR)
# https://en.wikipedia.org/wiki/Click-through_rate
stats['ctr'] = stats['click'] / stats['impressions'] * 100

# Calculate cost-per-mille (CPM) 
# - cost advertisers pay for 1k views or clicks
# https://en.wikipedia.org/wiki/Cost_per_mille
stats['cpm'] = stats['payprice'] / stats['impressions']

# Calculate effective cost-per-click (eCPC) 
# Cost to advertiser / number of clicks
#stats['ecpc'] = stats['payprice'] / stats['click']
ecpc = stats['payprice'] / stats['click']
stats['ecpc'] = ecpc / 1000


#######

#calculate constant bidding
stats['avg'] = stats['bidprice']/ stats['impressions']

#ccalculate total bidprice
total1 = stats['bidprice'].sum()
print ('Total bidprice=',total1)

#calculate total impression
total2 = stats['impressions'].sum()
print ('Total impressions=',total2)

#calculate constant bidding
cons = total1/total2
print('Constant bidding=',cons)

stats

Total bidprice= 663565548
Total impressions= 2430981
Constant bidding= 272.9620461862927


,click,bidprice,payprice,impressions,ctr,cpm,ecpc,avg
advertiser,,,,,,,,
1458,385,147705900,33968736,492353,0.078196,68.992646,88.230483,300.000000
2259,43,38508376,12428238,133673,0.032168,92.974931,289.028791,288.078939
2261,36,31701699,9873779,110122,0.032691,89.662184,274.271639,287.877981
2821,131,61343675,18828044,211366,0.061978,89.077922,143.725527,290.224894
2997,217,13802633,3129267,49829,0.435489,62.800116,14.420585,277.000000
3358,202,61729738,22447231,264956,0.076239,84.720599,111.124906,232.981091
3386,320,136512300,34931823,455041,0.070323,76.766320,109.161947,300.000000
3427,272,95218860,30458711,402806,0.067526,75.616329,111.980555,236.388882
3476,187,77042367,23918779,310835,0.060161,76.950083,127.907909,247.856152


In [31]:
uniq_fields = [ x for x in (ds_train) if x not in list(ds_test)]

data = ds_val
print('Fields in training/validation, not in test:', uniq_fields)

print(f'''---Mean---
{data[uniq_fields].mean()}
---Max---
{data[uniq_fields].max()}
---Min---
{data[uniq_fields].min()}
''')

Fields in training/validation, not in test: ['click', 'bidprice', 'payprice']
---Mean---
click         0.000665
bidprice    273.053099
payprice     78.234005
dtype: float64
---Max---
click         1
bidprice    300
payprice    300
dtype: int64
---Min---
click         0
bidprice    227
payprice      0
dtype: int64



## Validation Data

In [32]:
print('headers:', list(ds_val))
print('size:', len(ds_val))
ds_val.head()

headers: ['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag']
size: 303925


,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,4,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,windows_ie,211.144.203.*,79,79,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,23,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10111"
1,0,1,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,windows_chrome,58.247.250.*,79,79,1.0,...,90,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,75,a3becb967c015e9acb5e72f22b100ef8,3476,"10075,10057,10024,10052,10083,10063,10006,10110"
2,0,4,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,windows_ie,117.12.111.*,2,2,2.0,...,250,2,0,5,911b2d84826786018761e8c0b0a3a60c,238,65,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10024,10059,10063,13776,10083,10006,10111"
3,0,5,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,windows_chrome,61.187.224.*,201,205,2.0,...,280,2,0,5,dc0998c10f8f0b623b5d949e8272e4c7,238,6,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10057,10006,10063,10110"
4,0,1,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,windows_chrome,117.41.145.*,134,135,2.0,...,200,1,0,5,23485fcd23122d755d38f8c89d46ca56,249,5,7e0e909ea89f707bc20f9585c9072081,3476,"10006,10063,13776,10110"


In [33]:
df = ds_val

# Joining the 2 dataframes
stats = tmp1.join(tmp2.set_index('advertiser'), on='advertiser')

# Calculate Click-Through Rate (CTR)
# https://en.wikipedia.org/wiki/Click-through_rate
stats['ctr'] = stats['click'] / stats['impressions'] * 100

# Calculate cost-per-mille (CPM) - cost advertisers pay for 1k views or clicks
# https://en.wikipedia.org/wiki/Cost_per_mille
stats['cpm'] = stats['payprice'] / stats['impressions']

# Calculate effective cost-per-click (eCPC) 
# Cost to advertiser / number of clicks
stats['ecpc'] = stats['payprice'] / stats['click'] / 1000



stats

,click,bidprice,payprice,impressions,ctr,cpm,ecpc
advertiser,,,,,,,
1458,385,147705900,33968736,492353,0.078196,68.992646,88.230483
2259,43,38508376,12428238,133673,0.032168,92.974931,289.028791
2261,36,31701699,9873779,110122,0.032691,89.662184,274.271639
2821,131,61343675,18828044,211366,0.061978,89.077922,143.725527
2997,217,13802633,3129267,49829,0.435489,62.800116,14.420585
3358,202,61729738,22447231,264956,0.076239,84.720599,111.124906
3386,320,136512300,34931823,455041,0.070323,76.766320,109.161947
3427,272,95218860,30458711,402806,0.067526,75.616329,111.980555
3476,187,77042367,23918779,310835,0.060161,76.950083,127.907909


## Test Data

In [34]:
print('headers:', list(ds_test))
print('size:', len(ds_test))
ds_test.head()

headers: ['weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'keypage', 'advertiser', 'usertag']
size: 303375


,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,...,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,keypage,advertiser,usertag
0,0,12,366c563de7d90feb9d4dab53e795a93fb3157387,75045dd2f2136c93fe55fe6c446ec1527ed8f0bb,windows_chrome,27.197.36.*,146,159,3.0,DDTSQuf0MTTNaqKIvMpENpn,...,Astro_F_Rectangle,300,250,0,0,10,00fccc64a1ee2809348509b7ac2a97a5,b2e35064f3549d447edbbdfb1f707c8c,3427,"10024,10077,10075,10063,10031,10102,10006,1304..."
1,3,14,29167d4caa719788b5a342dbaa25151d53121f80,11279eb1f8f7a88f877db911673522b6ff202aa7,android_safari,124.126.227.*,1,1,NaN,d63d96468f66986d5a8eb5225ad0b6f0,...,2745306850,320,50,FirstView,Na,5,11908,NaN,2997,NaN
2,5,19,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,4771a819a3f5b86776d8a9456f4f2506578f78d8,windows_ie,116.116.104.*,27,34,1.0,trqRTJkrBoq7JsNr5SqfNX,...,mm_34022157_3445226_11175100,336,280,2,1,0,77819d3e0b3467fe5c7b16d68ad923a1,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10006,10024,10059,10048,10063,10067,1008..."
3,0,21,844c2da00d45315f20b748ec131c26ee99a7cbc7,5360671379f3204afa9cc0edd8fd2e2096cb09eb,windows_ie,115.46.145.*,238,245,4.0,6d27caf5a202ec085f80af7163ef7cb0,...,210842,960,90,FirstView,Na,30,10717,NaN,2821,"10075,13042,10006,10110,13776,10031,10063"
4,2,20,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,d791b5f6306b9d299f81daa5448ced7e5bfa010f,windows_ie,221.199.203.*,27,35,3.0,3FKElpuEMusyJqKbuKz,...,Fashion_F_Rectangle,300,250,0,0,50,44966cc8da1ed40c95d59e863c8c75f0,43f4a37f42a7c5e6219e2601b26c6976,3386,10063
